In [1]:
import os

In [2]:
# check the folder
%pwd

'e:\\Chicken_DIsease\\Chicken_Disease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Chicken_DIsease\\Chicken_Disease_Classification'

In [5]:
import tensorflow as tf

In [6]:
# Load the model from artifacts
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
# Define Evaluation Configuration

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    

In [8]:
from Chicken_Disease_Classification.constrants import * # Import Everything
from Chicken_Disease_Classification.utils.common import read_yaml, create_directories , save_json

In [9]:
# Config Manager
# Paste in Configuration (src\Chicken_Disease_Classification\config\configuration.py)

class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,  # Return Box Type  # Ctrl+click to check the file path
            params_filepath=PARAMS_FILE_PATH):

            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)

            # From common.py
            create_directories([self.config.artifacts_root]) # I can call using the key name using Box Type
    

    def get_validation_config(self) -> EvaluationConfig:
          eval_config=EvaluationConfig(
                path_of_model="artifacts/training/model.h5",
                training_data="artifacts/data_ingestion/Chicken-fecal-images",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE)
          return eval_config


In [10]:
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config

    def _valid_generator(self):

        datagenerator_kwargs=dict(
            rescale=1./255,     # Normalization
            validation_split=0.20
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1], # [224, 224]
            batch_size=self.config.params_batch_size,
            interpolation="bilinear" # Fill the gaps
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model: # Load the model
        return tf.keras.models.load_model(path)
    
    def evaluation(self):  # Evaluation
        self.model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)

    def save_score(self):     # Save the score in json format
        scores={"Loss": self.score[0], "accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

    

In [12]:
try:
    config=ConfigurationManager()
    val_config=config.get_validation_config()
    evaluation=Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e :
    raise e

# Value will be saved in scores.json

[2024-09-01 14:51:16,096: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-01 14:51:16,100: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-01 14:51:16,100: INFO: common: created directory at: artifacts]
Found 78 images belonging to 2 classes.
5/5 [==============================] - 19s 4s/step - loss: 0.5449 - accuracy: 0.9487
[2024-09-01 14:51:36,028: INFO: common: json file saved at: scores.json]
